#### credentials

In [ ]:
IMAP_SERVER = 'imap.gmail.com'
EMAIL_ACCOUNT = 'youremail@gmail.com' #your email
PASSWORD = 'xxxx xxxx xxxx xxxx' # your_app_password. Create here (https://myaccount.google.com/apppasswords)
COUNT_DAYS_AGO = 5

In [ ]:
import imaplib
import email
from email.header import decode_header
from collections import Counter
from datetime import datetime, timedelta
import ssl

#### Functions

In [ ]:
def connect_to_mail():
    context = ssl.create_default_context()
    mail = imaplib.IMAP4_SSL(IMAP_SERVER, ssl_context=context)
    mail.login(EMAIL_ACCOUNT, PASSWORD)
    mail.select('inbox')
    return mail


In [ ]:
def process_email(email_id, mail):
    try:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result == 'OK':
            raw_email = msg_data[0][1]
            msg = email.message_from_bytes(raw_email)
            
            from_header = msg['From']
            decoded_header = decode_header(from_header)
            for decoded_string, charset in decoded_header:
                if isinstance(decoded_string, bytes):
                    charset = charset or 'utf-8'
                    try:
                        decoded_string = decoded_string.decode(charset)
                    except LookupError:
                        decoded_string = decoded_string.decode('utf-8', errors='ignore')
                sender_email = email.utils.parseaddr(decoded_string)[1]
                if sender_email:
                    return sender_email
    except imaplib.IMAP4.abort as e:
        print(f"Error fetching email {email_id}: {e}")
    except Exception as e:
        print(f"Unexpected error fetching email {email_id}: {e}")
    return None


In [ ]:
def fetch_emails_since(mail, date_since):
    result, data = mail.search(None, f'SINCE {date_since}')
    if result == 'OK':
        return data[0].split()
    else:
        print('Error searching emails')
        return []

#### Connect to the server

In [ ]:

mail = connect_to_mail()

#### Define the date to limit the email search (e.g., emails from COUNT_DAYS_AGO)

In [ ]:

date_since = (datetime.now() - timedelta(days=COUNT_DAYS_AGO)).strftime("%d-%b-%Y")


#### Search for emails received after a certain date

In [ ]:

email_ids = fetch_emails_since(mail, date_since)


In [ ]:
senders = []

#### Process each email

In [ ]:
for email_id in email_ids:
    sender_email = process_email(email_id, mail)
    if sender_email:
        senders.append(sender_email)

#### Count the number of emails from each sender

In [ ]:

sender_counts = Counter(senders)

#### Display the results

In [ ]:

for sender, count in sender_counts.items():
    print(f'{sender}: {count} emails')
# Close the connection
mail.logout()